In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import io
from scipy import stats
import pickle
import glob
import pandas as pd
import math

#for mat file 
import scipy.io as sio
import seaborn as sb

# import user defined functions
from functions import *
from utils import * 

# Import decoder functions
from decoders import WienerCascadeDecoder
from decoders import WienerFilterDecoder
from decoders import DenseNNDecoder
from decoders import SimpleRNNDecoder
from decoders import GRUDecoder
from decoders import LSTMDecoder
from decoders import XGBoostDecoder
from decoders import SVRDecoder
from decoders import KalmanFilterDecoder
from sklearn.linear_model import LinearRegression

%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR

from itertools import product

/Users/Murano/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def make_mat_file(res_all,result_file):

    #save decoding result in .mat format
    mat_name='./mat_data/{}'.format(result_file.split('/')[-2])
    make_dir(mat_name)
    for k in range(len(res_all)):
        temp=res_all[k]
        mn=temp['model']
        dn=temp['data_name']
        save_name='{}/{}_{}'.format(mat_name,dn,mn)
        sio.savemat(save_name,temp)

In [3]:
def model_eval_not_flat(model):
    # by using not flat inputs, this function performs training, prediction, and evaluation
    # for RNN,GRU,LSTM
    
    #train
    model.fit(X_train, y_train)
    #prediction
    y_test_predicted=model.predict(X_test)
    y_test_predicted=y_test_predicted * y_train_std + y_train_mean
    #evaluation
    corr,R2,mae,y_test_predicted_smooth,corr_smooth,R2_smooth,mae_smooth=calc(y_test,y_test_predicted)
    #plot
    if plo==1:
        plot_all(y_test,y_test_predicted,out,data_name,model_name,target_name)
        
    res={'data_name':data_name,
         'y_test':y_test,
         'y_test_predicted':y_test_predicted,
         'corr':corr,
         'R2':R2,
         'mae':mae,
         'y_test_predicted_smooth':y_test_predicted_smooth,
         'corr_smooth':corr_smooth,
         'R2_smooth':R2_smooth,
         'mae_smooth':mae_smooth,
         'model':model_name,
         'n_neuron':n_neuron}

    return res

In [4]:
def velocity2(position):
    lenposi=len(position)
    v_cm_s=[]
    actual_posi=[]
    v_m=[];
    actual_posi=position*(40.0/200.0)
    for k in range(lenposi):
        tmp=0
        
        for i in range(-4,3): #313 filter
            if (k+i <= 0 or k+i+1 >= lenposi):
                tmp=tmp
            else:
                xaya = actual_posi[k+i+1] - actual_posi[k+i]             
                dist = np.sqrt(np.sum(xaya**2))
                tmp=tmp+dist
        v_cm_s.append(tmp*(3.0/7.0)) #313 filter
    v_m=np.array(v_cm_s)
    
    return  v_m

In [5]:
def head3(position):#tangent value
    lenposi=len(position)
    actual_posi=[]
    head_posi_s=[]
    head_posi_s2=[]
    head_posi_s3=[]
    head_posi=[]
    actual_posi=position*(40.0/200.0)
    for k in range(lenposi):
        if(k <=2 or k >= lenposi-4):
            tmp=0
        else:
            xaya=actual_posi[k+3]-actual_posi[k-3]
            xa=xaya[0]
            ya=xaya[1]        
            tmp=math.atan2(ya,xa)
        tmp=tmp+math.pi/2.0
        head_posi_s.append(tmp)
    head_posi_s2=np.array(head_posi_s)
    headlen=len(head_posi_s2)
    for k in range(headlen):
        if (head_posi_s[k] > math.pi):
            tmp2=head_posi_s[k]-2*math.pi
        else:
            tmp2=head_posi_s[k]
        head_posi_s3.append(tmp2)
    #from IPython.core.debugger import Pdb; Pdb().set_trace()
    head_posi=np.array(head_posi_s3)
    #print head_posi
    return head_posi 

In [6]:
def v_th2(position,th):#Use this function if you want to run decoding when mice are running.
    vm = velocity2(position)
    use_time_v=vm>=th
    plt.plot(vm)
    print(np.sum(use_time_v))
    return use_time_v

In [7]:
def v_th3(position,th):#Use this function if you want to run decoding when mice are stopping.
    vm = velocity2(position)
    use_time_v=vm<th
    plt.plot(vm)
    print(np.sum(use_time_v))
    return use_time_v

In [8]:
velocity_th_list=[1.0]#Velocity Filter cm/sec
using_neuron_rate_list = [0, 0.5] #0: all neurons will be used, 0.25: 75% of neurons, 0.5: half, 0.75: 25% of neurons 
info_list= [0] #Specify the Deletion order #0: actual index, 1: negacon index
# to change the deletion order, modify the name of information list in function.py. please see line 139 of function.py.

#Note that when you use negacon=2 and neurate, 0:all neurons, 0.25: 25% of neurons, 0.5: half, 0.75:75% of neurons
#######################
#result output directory


for th_velo, info_ind, neurate in product(velocity_th_list, info_list, using_neuron_rate_list):
    out='./result_Pos_neurate_{}_vf_{}'.format(neurate, th_velo)
    make_dir(out)

    result_dir=out#dir of decoding result
    result_file='{}/res_all.pkl'.format(result_dir)
    
    data_dir='./data/alldata//'
    
    #choose data you want to use for decoding
    datalist_all=glob.glob('{}/*'.format(data_dir))
    

    plo=0 # set 1, if you want to plot the decoding result
    negacon=0 # set 1, if you want to shuffle the data and make negative control; set 2, if you want to calculate negacon data for neurate.
    zscore=0 # set 1, if you want to standarlize the position for machine learning models
 
    
    ###hyper parameters
    #for kalman filter
    lag=10
    c=1
    #for PF
    sig_PF=10
    #for neural networks
    epoch=10
    unit=1000
    dropout=0.1
    
    
    #######################
    np.random.seed(100)

    res_all=[]
    pf_score_all=[]
    for datalist_name in datalist_all:
        print(datalist_name)
        datalist=glob.glob(datalist_name + '/*')
        data_name=datalist[0].split('/')[-2]

        #load data
        info, trace,spike,position,x_position,y_position,n_neuron,n_time,firing_rate,trace_smooth=data_load3(datalist)
        
        if 'cam30' in data_dir.split('/')[2]:# if data used is camkii hko data of 60 minutes, I use first 30 minutes
            trace=trace[:5395,:]
            spike=spike[:5395,:]
            position=position[:5395,:]
            x_position=x_position[:5395]
            y_position=y_position[:5395]
            n_time=5395
            firing_rate=firing_rate[:5395,:]
            trace_smooth=trace_smooth[:5395,:]

        #thresholding by velocity
        use_time_v=v_th2(position,th_velo)
       
        trace=trace[use_time_v,:]
        spike=spike[use_time_v,:]
        position=position[use_time_v,:]
        x_position=x_position[use_time_v]
        y_position=y_position[use_time_v]
        n_time=np.sum(use_time_v)
        firing_rate=firing_rate[use_time_v,:]
        trace_smooth=trace_smooth[use_time_v,:]
        
        # I dont use these options, but original code contain these options and I leave it
        bins_before=0 #How many bins of neural data prior to the output are used for decoding
        bins_current=1 #Whether to use concurrent time bin of neural data
        bins_after=0 #How many bins of neural data after the output are used for decoding

        ## Set what part of data should be part of the training/testing sets
        train_range=[0, 0.5]
        test_range=[0.5, 1.0]

        train_set=np.arange(np.int(np.round(train_range[0]*n_time))+bins_before,np.int(np.round(train_range[1]*n_time))-bins_after)
        test_set=np.arange(np.int(np.round(test_range[0]*n_time))+bins_before,np.int(np.round(test_range[1]*n_time))-bins_after)
            
        use_data=np.sum(spike[train_set,:],axis=0)>0#remove the neuron which dont activate at training
        spike=spike[:,use_data]
        trace=trace[:,use_data]
        firing_rate=firing_rate[:,use_data]
        trace_smooth=trace_smooth[:,use_data]
        info=info[use_data]

        n_neuron=spike.shape[1]
            
        firing_rate_for_score=firing_rate
        if negacon==1:# in the case of negacon, I randomly reorder spike sequence for training
            for k in range(n_neuron):
                r=np.random.permutation(len(train_set))
                spike[train_set,k]=spike[train_set,k][r]
            firing_rate=estimate_firing_rate(spike)

            for k in range(n_neuron):
                r=np.random.permutation(n_time)
                spike[:,k]=spike[r,k]
            firing_rate_for_score=estimate_firing_rate(spike)


        #####
        neural_data=firing_rate # neural activity data used in decoding
    
        if negacon == 2:
            [neural_data,n_neuron] = neuron_thresholding3(neural_data,position,train_set,neurate)
        else:
            [neural_data,n_neuron,info_ind] = neuron_thresholding2(neural_data,position,train_set,neurate,info,info_ind)
            
        neural_data=neural_data-np.mean(neural_data,axis=0,keepdims=True)
        neural_data_for_score=firing_rate_for_score # neural activity data used in calculating correlation between neural activity and position
        neural_data_for_score=neural_data_for_score-np.mean(neural_data_for_score,axis=0,keepdims=True)
        #####

        X=get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)
        X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

        ######################## choose the types of decoding, position, speed, or motion direction.
        target_name = ['x_position','y_position'] #target variable name for plot
        target_is_position = True
        y=position[:] #target variable. shape is (n_time , 2)
        
        #target_name = ['head direction'] #target variable name for plot
        #y=head3(position).reshape(-1,1) #target variable. shape is (n_time , 1)
        #target_is_position = False
        
        #target_name = ['speed'] #target variable name for plot
        #y=velocity2(position).reshape(-1,1) #target variable. shape is (n_time , 1)
        #target_is_position = False
        ########################

        ### 3C. Split into training / testing / testation sets

        #Get training data
        X_train=X[train_set,:,:]
        X_flat_train=X_flat[train_set,:]
        y_train=y[train_set,:]

        #Get testing data
        X_test=X[test_set,:,:]
        X_flat_test=X_flat[test_set,:]
        y_test=y[test_set,:]

        #Z-score "X" inputs. 
        X_train_mean=np.nanmean(X_train,axis=0)
        X_train_std=np.nanstd(X_train,axis=0)
        X_train=(X_train-X_train_mean)/X_train_std
        X_test=(X_test-X_train_mean)/X_train_std

        #Z-score "X_flat" inputs. 
        X_flat_train_mean=np.nanmean(X_flat_train,axis=0)
        X_flat_train_std=np.nanstd(X_flat_train,axis=0)
        X_flat_train=(X_flat_train-X_flat_train_mean)/X_flat_train_std
        X_flat_test=(X_flat_test-X_flat_train_mean)/X_flat_train_std

        #Zero-center or zscore outputs
        y_train_mean=np.mean(y_train,axis=0)
        y_train_std=np.std(y_train,axis=0)
        y_train_std_svr=np.std(y_train,axis=0)
        if zscore==0:
            y_train_std=y_train_std*0 +1
        y_train_svr=(y_train-y_train_mean)/y_train_std_svr#data for svr is standarized if zscore =0
        y_train=(y_train-y_train_mean)/y_train_std

        ## 4. Run Decoders-----------------------------------------------------------------------
         

        ### 4H. LSTM (Long Short Term Memory)
        model_name='lstm'
        model=LSTMDecoder(units=unit,dropout=dropout,num_epochs=epoch)
        res=model_eval_not_flat(model)
        res_all.append(res)
        print('{} : corr : {}'.format(model_name,res['corr_smooth']))
 

        plt.close('all')
        print('----')

    #ensemble
    df=pd.DataFrame(res_all)

 


    with open('{}/res_all.pkl'.format(out),'wb') as f:
        pickle.dump(res_all,f)

    result={#'res_all':res_all,
            'bins_after':bins_after,
            'bins_current':bins_current,
            'bins_before':bins_before,
            'epoch':epoch,
            'dropout':dropout,
            'unit':unit,
            'negacon':negacon,
            'C':c,
            'lag':lag,
            'sig_PF':sig_PF,
            'zscore':zscore,
            'PF_score':pf_score_all,
            'th_velo':th_velo,
            'neurate':neurate}

    with open('{}/parameters.pkl'.format(out),'wb') as f:
        pickle.dump(result,f)
        
    ##Generate mat file
    make_mat_file(res_all,result_file)


mkdir : ./result_Pos_neurate_0_vf_1.0
./data/alldata/092717 OF SERT WT M32-n1
2180
79 neurons are used
mkdir : ./result_Pos_neurate_0_vf_1.0/figure_lstm
lstm : corr : [0.6806690066313473, 0.6880130781231317]
----
./data/alldata/092217 OF CaMKII HKO M30-n1
4054
59 neurons are used
lstm : corr : [0.21678641785410918, 0.24860366583758525]
----
./data/alldata/091317 OF CaMKII HKO M20-n1
2921
80 neurons are used
lstm : corr : [-0.13482203644745888, 0.2206340709799211]
----
./data/alldata/090817 OF CaMKII HKO M22-n1
2893
69 neurons are used
lstm : corr : [0.3449884946203825, 0.4807512298557625]
----
./data/alldata/092217 OF CaMKII WT M29-n1
2269
63 neurons are used
lstm : corr : [0.6507311423389146, 0.5673733067382011]
----
./data/alldata/M46_042718_OF
3072
89 neurons are used
lstm : corr : [0.787377231931056, 0.7600616100237094]
----
./data/alldata/081117 OF B6J M27-n1
3379
57 neurons are used
lstm : corr : [0.8096559065141203, 0.7538623364638083]
----
./data/alldata/091317 OF CaMKII HKO M1